In [ ]:
import os
from pyspark import SparkContext, SQLContext
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [ ]:
from pyspark.sql.types import StringType, FloatType, ArrayType

In [ ]:
spark = SparkSession \
    .builder \
    .master('local[*]') \
    .getOrCreate()

sc = spark.sparkContext

In [ ]:
#files_path_bio = "/data/biorxiv_medrxiv/biorxiv_medrxiv/"
files_path_bio = "/data/test_data/"

In [ ]:
list_files = os.listdir(files_path_bio)
input_files = spark.read.option("multiline",True).json(files_path_bio)

In [ ]:
spark

In [ ]:
input_files.take(1)

In [ ]:
input_files.schema

In [ ]:
input_files.show(1)

In [ ]:
input_files.columns

In [ ]:
input_files.printSchema()

In [ ]:
input_files.select('paper_id').take(1)

In [ ]:
input_files.abstract

In [ ]:
input_files.select(F.col('abstract').getField('text')[0]).show()

In [ ]:
input_files.select(F.col('body_text')[0].getField('text')).show()

In [ ]:
input_files.select(F.col('body_text').getField('text')[0]).show()

In [ ]:
input_files.select(F.col('body_text')[0].getField('text')).collect()[0][0]

In [ ]:
input_files.select(F.col('body_text')).take(1)

In [ ]:
input_files.select('body_text').printSchema()

In [ ]:
def format_body(body_text):
    texts = [(di['section'], di['text']) for di in body_text]
    texts_di = {di['section']: "" for di in body_text}
    
    for section, text in texts:
        texts_di[section] += text

    body = ""

    for section, text in texts_di.items():
        body += section
        body += "\n\n"
        body += text
        body += "\n\n"
    
    return body

In [ ]:
body_f = F.udf(format_body)

In [ ]:
print(input_files.withColumn('text_body', body_f(F.col('body_text'))).select('text_body').collect()[0][0])

In [ ]:
@F.udf()
def format_body(body_text):
    texts = [(di['section'], di['text']) for di in body_text]
    texts_di = {di['section']: "" for di in body_text}
    
    for section, text in texts:
        texts_di[section] += text

    body = ""

    for section, text in texts_di.items():
        body += section
        body += "\n\n"
        body += text
        body += "\n\n"
    
    return body

In [ ]:
print(input_files.withColumn('text_body2', format_body(F.col('body_text'))).select('text_body2').collect()[0][0])